In [ ]:
import subprocess
video_pth="../Train/anomaly/2.mp4"
output_pth="../Train/anomaly/output2.h264"
res_pth="../Train/anomaly/res2.txt"
command = ['ffmpeg', '-i', video_pth, '-c:v', 'libx264', output_pth]

try:
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
except subprocess.CalledProcessError as e:
    print("FFmpeg returned an error:", e.stderr)

In [4]:
import subprocess

command = ["ffprobe", "-select_streams", "v:0", "-show_frames", "-print_format", "json", output_pth]

with open(res_pth, "w") as file:
    subprocess.call(command, stdout=file)